In [2]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
import keras
from keras.layers import Dropout

# Reading the data
train = pd.read_csv('./drive/My Drive/Datasets/house_predict.csv')
trainX, trainY = train.iloc[:, :train.shape[1]-1], train.iloc[:, train.shape[1]-1]
# There are a total of 43 categorical columns
categoricals = trainX.loc[:, trainX.dtypes == 'O'].columns
len(categoricals) # 43
# Preprocessing step: 
# One Hot Encoder cannot work with NaN, hence filling NaN with mode of categorical columns
cat_features = trainX.loc[:, categoricals]
cat_features = cat_features.fillna(cat_features.mode().iloc[0, :])
# One hot encoding these features
ohe = OneHotEncoder(handle_unknown='ignore')
res = ohe.fit_transform(cat_features).toarray()
cols = np.array([])
for i in range(cat_features.shape[1]):
    cols = np.concatenate((cols, categoricals[i] + '_' + np.sort(cat_features.iloc[:, i].unique())))    
cat = pd.DataFrame(res, columns=cols)
# Total 252 categorical features
cat.shape # (1460, 252)
# Dropping original categorical variables
trainX = trainX.drop(categoricals, axis=1)
# Concatenating the One Hot Encoded variables to the train dataset
trainX = pd.concat([trainX, cat], axis=1)
# New data shape
trainX.shape # (1460, 289)
# Filling the NaN with median
trainX.fillna(trainX.median(), inplace=True)
# Normalizing training features
scalar = MinMaxScaler()
norm_train = pd.DataFrame(scalar.fit_transform(trainX), columns=trainX.columns)
# Normalizing training target
scalar_target = MinMaxScaler()
trainY = scalar_target.fit_transform(trainY.values.reshape(-1, 1))
# Defining the network
model = Sequential([
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='relu'),
  Dropout(0.30),
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='relu'),
  Dropout(0.30),
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='relu'),
  Dropout(0.30),
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='relu'),
  Dropout(0.30),
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='relu'),
  Dropout(0.30),      
  # Dense(units=norm_train.shape[1]//2, activation='sigmoid'),    
  Dense(units=1, activation='softmax'),
])
# Printing model summary
model.summary()
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# dense_34 (Dense)             (None, 289)               83810     
# _________________________________________________________________
# dense_35 (Dense)             (None, 144)               41760     
# _________________________________________________________________
# dense_36 (Dense)             (None, 1)                 145       
# =================================================================
# Total params: 125,715
# Trainable params: 125,715
# Non-trainable params: 0
# Compiling and Training Network
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(trainX, trainY, batch_size=512, epochs=21, verbose=1, validation_split=0.2)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 289)               83810     
_________________________________________________________________
dropout_1 (Dropout)          (None, 289)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 289)               83810     
_________________________________________________________________
dropout_2 (Dropout)          (None, 289)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 289)               83810     
_________________________________________________________________
dropout_3 (Dropout)          (None, 289)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 289)              